In [21]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file

city_data_df = pd.read_csv("/Users/maron/Class/World_Weather_Analysis/Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Chuy,UY,-33.6971,-53.4616,48.92,83,6,9.28,clear sky
1,1,Vila Franca Do Campo,PT,37.7167,-25.4333,72.01,88,29,6.69,light rain
2,2,Ponta Do Sol,PT,32.6667,-17.1000,72.72,81,1,3.42,clear sky
3,3,Port Alfred,ZA,-33.5906,26.8910,62.19,97,100,5.01,light rain
4,4,Busselton,AU,-33.6500,115.3333,57.33,75,100,21.47,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria

min_temp = float(input("What is the minimum temperature you would like for the trip? "))
max_temp = float(input("What is the maximum temperature you would like for the trip? "))

What is the minimum temperature you would like for the trip? 70
What is the maximum temperature you would like for the trip? 85


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                   (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Vila Franca Do Campo,PT,37.7167,-25.4333,72.01,88,29,6.69,light rain
2,2,Ponta Do Sol,PT,32.6667,-17.1000,72.72,81,1,3.42,clear sky
7,7,Wawa,NG,9.9022,4.4192,74.77,85,100,5.97,overcast clouds
8,8,Pangai,TO,-19.8000,-174.3500,77.49,79,41,15.61,scattered clouds
12,12,Kapaa,US,22.0752,-159.3190,84.18,77,100,11.50,overcast clouds


In [23]:
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Vila Franca Do Campo,PT,37.7167,-25.4333,72.01,88,29,6.69,light rain
2,2,Ponta Do Sol,PT,32.6667,-17.1000,72.72,81,1,3.42,clear sky
7,7,Wawa,NG,9.9022,4.4192,74.77,85,100,5.97,overcast clouds
8,8,Pangai,TO,-19.8000,-174.3500,77.49,79,41,15.61,scattered clouds
12,12,Kapaa,US,22.0752,-159.3190,84.18,77,100,11.50,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
665,665,Moroni,KM,-11.7022,43.2551,73.58,76,5,6.40,clear sky
670,670,Sabzevar,IR,36.2126,57.6819,81.68,26,0,20.71,clear sky
672,672,El Sauzal,MX,31.9000,-116.6833,74.41,82,84,7.34,broken clouds
675,675,Roquetas De Mar,ES,36.7642,-2.6148,82.62,71,0,4.23,clear sky


In [6]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.count()

City_ID                259
City                   259
Country                259
Lat                    259
Lng                    259
Max Temp               259
Humidity               259
Cloudiness             259
Wind Speed             259
Current Description    259
dtype: int64

In [ ]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

# N/A no empty rows detected

In [17]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Vila Franca Do Campo,PT,72.01,light rain,37.7167,-25.4333,
2,Ponta Do Sol,PT,72.72,clear sky,32.6667,-17.1000,
7,Wawa,NG,74.77,overcast clouds,9.9022,4.4192,
8,Pangai,TO,77.49,scattered clouds,-19.8000,-174.3500,
12,Kapaa,US,84.18,overcast clouds,22.0752,-159.3190,
13,Araruama,BR,73.35,light rain,-22.8728,-42.3431,
15,Wewak,PG,78.62,overcast clouds,-3.5534,143.6268,
17,Mizdah,LY,78.21,clear sky,31.4451,12.9801,
29,Jalu,LY,82.90,clear sky,29.0331,21.5482,
31,Chateaubelair,VC,81.39,clear sky,13.2833,-61.2500,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
        
# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]    
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel not found... skipping")
        

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [19]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Vila Franca Do Campo,PT,72.01,light rain,37.7167,-25.4333,Pestana Bahia Praia
2,Ponta Do Sol,PT,72.72,clear sky,32.6667,-17.1000,Hotel do Campo
7,Wawa,NG,74.77,overcast clouds,9.9022,4.4192,
8,Pangai,TO,77.49,scattered clouds,-19.8000,-174.3500,Ha'apai Beach Resort
12,Kapaa,US,84.18,overcast clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort Hotel


In [24]:
# 7. Drop the rows where there is no Hotel Name.

hotel_df = hotel_df.replace("", np.NaN, regex=True)

hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Vila Franca Do Campo,PT,72.01,light rain,37.7167,-25.4333,Pestana Bahia Praia
2,Ponta Do Sol,PT,72.72,clear sky,32.6667,-17.1000,Hotel do Campo
7,Wawa,NG,74.77,overcast clouds,9.9022,4.4192,NaN
8,Pangai,TO,77.49,scattered clouds,-19.8000,-174.3500,Ha'apai Beach Resort
12,Kapaa,US,84.18,overcast clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort Hotel
...,...,...,...,...,...,...,...
665,Moroni,KM,73.58,clear sky,-11.7022,43.2551,Hotel Les Arcades
670,Sabzevar,IR,81.68,clear sky,36.2126,57.6819,پزشکی قانونی سبزوار
672,El Sauzal,MX,74.41,broken clouds,31.9000,-116.6833,Hotel Las Rosas & Spa
675,Roquetas De Mar,ES,82.62,clear sky,36.7642,-2.6148,Mediterráneo Bay Hotel and Resort


In [25]:
# 7. Drop the rows where there is no Hotel Name.

hotel_df = hotel_df.dropna()

hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Vila Franca Do Campo,PT,72.01,light rain,37.7167,-25.4333,Pestana Bahia Praia
2,Ponta Do Sol,PT,72.72,clear sky,32.6667,-17.1000,Hotel do Campo
8,Pangai,TO,77.49,scattered clouds,-19.8000,-174.3500,Ha'apai Beach Resort
12,Kapaa,US,84.18,overcast clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort Hotel
13,Araruama,BR,73.35,light rain,-22.8728,-42.3431,Apartamento Quality


In [26]:
# 8a. Create the output File (CSV)
output_data_file = "/Users/maron/Class/World_Weather_Analysis/Weather_Database/Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [27]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.

info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [28]:
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [29]:
# 10b. Get the latitude and longitude from each row and store in a new DataFrame.

locations = hotel_df[["Lat", "Lng"]]

In [33]:
# 11a. Add a marker layer for each city to the map. 

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))